In [41]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains common code shared by all inception models.

Usage of arg scope:
  with slim.arg_scope(inception_arg_scope()):
    logits, end_points = inception.inception_v3(images, num_classes,
                                                is_training=is_training)

"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import tf_slim as slim


def inception_arg_scope(
    weight_decay=0.00004,
    use_batch_norm=True,
    batch_norm_decay=0.9997,
    batch_norm_epsilon=0.001,
    activation_fn=tf.nn.relu,
    batch_norm_updates_collections=tf.GraphKeys.UPDATE_OPS,
    batch_norm_scale=False):
  """Defines the default arg scope for inception models.

  Args:
    weight_decay: The weight decay to use for regularizing the model.
    use_batch_norm: "If `True`, batch_norm is applied after each convolution.
    batch_norm_decay: Decay for batch norm moving average.
    batch_norm_epsilon: Small float added to variance to avoid dividing by zero
      in batch norm.
    activation_fn: Activation function for conv2d.
    batch_norm_updates_collections: Collection for the update ops for
      batch norm.
    batch_norm_scale: If True, uses an explicit `gamma` multiplier to scale the
      activations in the batch normalization layer.

  Returns:
    An `arg_scope` to use for the inception models.
  """
  batch_norm_params = {
      # Decay for the moving averages.
      'decay': batch_norm_decay,
      # epsilon to prevent 0s in variance.
      'epsilon': batch_norm_epsilon,
      # collection containing update_ops.
      'updates_collections': batch_norm_updates_collections,
      # use fused batch norm if possible.
      'fused': None,
      'scale': batch_norm_scale,
  }
  if use_batch_norm:
    normalizer_fn = slim.batch_norm
    normalizer_params = batch_norm_params
  else:
    normalizer_fn = None
    normalizer_params = {}
  # Set weight_decay for weights in Conv and FC layers.
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      weights_regularizer=slim.l2_regularizer(weight_decay)):
    with slim.arg_scope(
        [slim.conv2d],
        weights_initializer=slim.variance_scaling_initializer(),
        activation_fn=activation_fn,
        normalizer_fn=normalizer_fn,
        normalizer_params=normalizer_params) as sc:
      return sc

In [42]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains the definition of the Inception V4 architecture.

As described in http://arxiv.org/abs/1602.07261.

  Inception-v4, Inception-ResNet and the Impact of Residual Connections
    on Learning
  Christian Szegedy, Sergey Ioffe, Vincent Vanhoucke, Alex Alemi
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import tf_slim as slim


def block_inception_a(inputs, scope=None, reuse=None):
  """Builds Inception-A block for Inception v4 network."""
  # By default use stride=1 and SAME padding
  with slim.arg_scope([slim.conv2d, slim.avg_pool2d, slim.max_pool2d],
                      stride=1, padding='SAME'):
    with tf.compat.v1.variable_scope(
        scope, 'BlockInceptionA', [inputs], reuse=reuse):
      with tf.compat.v1.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(inputs, 96, [1, 1], scope='Conv2d_0a_1x1')
      with tf.compat.v1.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(inputs, 64, [1, 1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 96, [3, 3], scope='Conv2d_0b_3x3')
      with tf.compat.v1.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(inputs, 64, [1, 1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
        branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
      with tf.compat.v1.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(inputs, [3, 3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 96, [1, 1], scope='Conv2d_0b_1x1')
      return tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])


def block_reduction_a(inputs, scope=None, reuse=None):
  """Builds Reduction-A block for Inception v4 network."""
  # By default use stride=1 and SAME padding
  with slim.arg_scope([slim.conv2d, slim.avg_pool2d, slim.max_pool2d],
                      stride=1, padding='SAME'):
    with tf.compat.v1.variable_scope(
        scope, 'BlockReductionA', [inputs], reuse=reuse):
      with tf.compat.v1.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(inputs, 384, [3, 3], stride=2, padding='VALID',
                               scope='Conv2d_1a_3x3')
      with tf.compat.v1.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(inputs, 192, [1, 1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 224, [3, 3], scope='Conv2d_0b_3x3')
        branch_1 = slim.conv2d(branch_1, 256, [3, 3], stride=2,
                               padding='VALID', scope='Conv2d_1a_3x3')
      with tf.compat.v1.variable_scope('Branch_2'):
        branch_2 = slim.max_pool2d(inputs, [3, 3], stride=2, padding='VALID',
                                   scope='MaxPool_1a_3x3')
      return tf.concat(axis=3, values=[branch_0, branch_1, branch_2])


def block_inception_b(inputs, scope=None, reuse=None):
  """Builds Inception-B block for Inception v4 network."""
  # By default use stride=1 and SAME padding
  with slim.arg_scope([slim.conv2d, slim.avg_pool2d, slim.max_pool2d],
                      stride=1, padding='SAME'):
    with tf.compat.v1.variable_scope(
        scope, 'BlockInceptionB', [inputs], reuse=reuse):
      with tf.compat.v1.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(inputs, 384, [1, 1], scope='Conv2d_0a_1x1')
      with tf.compat.v1.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(inputs, 192, [1, 1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 224, [1, 7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 256, [7, 1], scope='Conv2d_0c_7x1')
      with tf.compat.v1.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(inputs, 192, [1, 1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 192, [7, 1], scope='Conv2d_0b_7x1')
        branch_2 = slim.conv2d(branch_2, 224, [1, 7], scope='Conv2d_0c_1x7')
        branch_2 = slim.conv2d(branch_2, 224, [7, 1], scope='Conv2d_0d_7x1')
        branch_2 = slim.conv2d(branch_2, 256, [1, 7], scope='Conv2d_0e_1x7')
      with tf.compat.v1.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(inputs, [3, 3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 128, [1, 1], scope='Conv2d_0b_1x1')
      return tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])


def block_reduction_b(inputs, scope=None, reuse=None):
  """Builds Reduction-B block for Inception v4 network."""
  # By default use stride=1 and SAME padding
  with slim.arg_scope([slim.conv2d, slim.avg_pool2d, slim.max_pool2d],
                      stride=1, padding='SAME'):
    with tf.compat.v1.variable_scope(
        scope, 'BlockReductionB', [inputs], reuse=reuse):
      with tf.compat.v1.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(inputs, 192, [1, 1], scope='Conv2d_0a_1x1')
        branch_0 = slim.conv2d(branch_0, 192, [3, 3], stride=2,
                               padding='VALID', scope='Conv2d_1a_3x3')
      with tf.compat.v1.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(inputs, 256, [1, 1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 256, [1, 7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 320, [7, 1], scope='Conv2d_0c_7x1')
        branch_1 = slim.conv2d(branch_1, 320, [3, 3], stride=2,
                               padding='VALID', scope='Conv2d_1a_3x3')
      with tf.compat.v1.variable_scope('Branch_2'):
        branch_2 = slim.max_pool2d(inputs, [3, 3], stride=2, padding='VALID',
                                   scope='MaxPool_1a_3x3')
      return tf.concat(axis=3, values=[branch_0, branch_1, branch_2])


def block_inception_c(inputs, scope=None, reuse=None):
  """Builds Inception-C block for Inception v4 network."""
  # By default use stride=1 and SAME padding
  with slim.arg_scope([slim.conv2d, slim.avg_pool2d, slim.max_pool2d],
                      stride=1, padding='SAME'):
    with tf.compat.v1.variable_scope(
        scope, 'BlockInceptionC', [inputs], reuse=reuse):
      with tf.compat.v1.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(inputs, 256, [1, 1], scope='Conv2d_0a_1x1')
      with tf.compat.v1.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(inputs, 384, [1, 1], scope='Conv2d_0a_1x1')
        branch_1 = tf.concat(axis=3, values=[
            slim.conv2d(branch_1, 256, [1, 3], scope='Conv2d_0b_1x3'),
            slim.conv2d(branch_1, 256, [3, 1], scope='Conv2d_0c_3x1')])
      with tf.compat.v1.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(inputs, 384, [1, 1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 448, [3, 1], scope='Conv2d_0b_3x1')
        branch_2 = slim.conv2d(branch_2, 512, [1, 3], scope='Conv2d_0c_1x3')
        branch_2 = tf.concat(axis=3, values=[
            slim.conv2d(branch_2, 256, [1, 3], scope='Conv2d_0d_1x3'),
            slim.conv2d(branch_2, 256, [3, 1], scope='Conv2d_0e_3x1')])
      with tf.compat.v1.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(inputs, [3, 3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 256, [1, 1], scope='Conv2d_0b_1x1')
      return tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])


def inception_v4_base(inputs, final_endpoint='Mixed_7d', scope=None):
  """Creates the Inception V4 network up to the given final endpoint.

  Args:
    inputs: a 4-D tensor of size [batch_size, height, width, 3].
    final_endpoint: specifies the endpoint to construct the network up to.
      It can be one of [ 'Conv2d_1a_3x3', 'Conv2d_2a_3x3', 'Conv2d_2b_3x3',
      'Mixed_3a', 'Mixed_4a', 'Mixed_5a', 'Mixed_5b', 'Mixed_5c', 'Mixed_5d',
      'Mixed_5e', 'Mixed_6a', 'Mixed_6b', 'Mixed_6c', 'Mixed_6d', 'Mixed_6e',
      'Mixed_6f', 'Mixed_6g', 'Mixed_6h', 'Mixed_7a', 'Mixed_7b', 'Mixed_7c',
      'Mixed_7d']
    scope: Optional compat.v1.variable_scope.

  Returns:
    logits: the logits outputs of the model.
    end_points: the set of end_points from the inception model.

  Raises:
    ValueError: if final_endpoint is not set to one of the predefined values,
  """
  end_points = {}

  def add_and_check_final(name, net):
    end_points[name] = net
    return name == final_endpoint

  with tf.compat.v1.variable_scope(scope, 'InceptionV4', [inputs]):
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                        stride=1, padding='SAME'):
      # 299 x 299 x 3
      net = slim.conv2d(inputs, 32, [3, 3], stride=2,
                        padding='VALID', scope='Conv2d_1a_3x3')
      if add_and_check_final('Conv2d_1a_3x3', net): return net, end_points
      # 149 x 149 x 32
      net = slim.conv2d(net, 32, [3, 3], padding='VALID',
                        scope='Conv2d_2a_3x3')
      if add_and_check_final('Conv2d_2a_3x3', net): return net, end_points
      # 147 x 147 x 32
      net = slim.conv2d(net, 64, [3, 3], scope='Conv2d_2b_3x3')
      if add_and_check_final('Conv2d_2b_3x3', net): return net, end_points
      # 147 x 147 x 64
      with tf.compat.v1.variable_scope('Mixed_3a'):
        with tf.compat.v1.variable_scope('Branch_0'):
          branch_0 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID',
                                     scope='MaxPool_0a_3x3')
        with tf.compat.v1.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, 96, [3, 3], stride=2, padding='VALID',
                                 scope='Conv2d_0a_3x3')
        net = tf.concat(axis=3, values=[branch_0, branch_1])
        if add_and_check_final('Mixed_3a', net): return net, end_points

      # 73 x 73 x 160
      with tf.compat.v1.variable_scope('Mixed_4a'):
        with tf.compat.v1.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
          branch_0 = slim.conv2d(branch_0, 96, [3, 3], padding='VALID',
                                 scope='Conv2d_1a_3x3')
        with tf.compat.v1.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, 64, [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, 64, [7, 1], scope='Conv2d_0c_7x1')
          branch_1 = slim.conv2d(branch_1, 96, [3, 3], padding='VALID',
                                 scope='Conv2d_1a_3x3')
        net = tf.concat(axis=3, values=[branch_0, branch_1])
        if add_and_check_final('Mixed_4a', net): return net, end_points

      # 71 x 71 x 192
      with tf.compat.v1.variable_scope('Mixed_5a'):
        with tf.compat.v1.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, 192, [3, 3], stride=2, padding='VALID',
                                 scope='Conv2d_1a_3x3')
        with tf.compat.v1.variable_scope('Branch_1'):
          branch_1 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID',
                                     scope='MaxPool_1a_3x3')
        net = tf.concat(axis=3, values=[branch_0, branch_1])
        if add_and_check_final('Mixed_5a', net): return net, end_points

      # 35 x 35 x 384
      # 4 x Inception-A blocks
      for idx in range(4):
        block_scope = 'Mixed_5' + chr(ord('b') + idx)
        net = block_inception_a(net, block_scope)
        if add_and_check_final(block_scope, net): return net, end_points

      # 35 x 35 x 384
      # Reduction-A block
      net = block_reduction_a(net, 'Mixed_6a')
      if add_and_check_final('Mixed_6a', net): return net, end_points

      # 17 x 17 x 1024
      # 7 x Inception-B blocks
      for idx in range(7):
        block_scope = 'Mixed_6' + chr(ord('b') + idx)
        net = block_inception_b(net, block_scope)
        if add_and_check_final(block_scope, net): return net, end_points

      # 17 x 17 x 1024
      # Reduction-B block
      net = block_reduction_b(net, 'Mixed_7a')
      if add_and_check_final('Mixed_7a', net): return net, end_points

      # 8 x 8 x 1536
      # 3 x Inception-C blocks
      for idx in range(3):
        block_scope = 'Mixed_7' + chr(ord('b') + idx)
        net = block_inception_c(net, block_scope)
        if add_and_check_final(block_scope, net): return net, end_points
  raise ValueError('Unknown final endpoint %s' % final_endpoint)


def inception_v4(inputs, num_classes=1001, is_training=True,
                 dropout_keep_prob=0.8,
                 reuse=None,
                 scope='InceptionV4',
                 create_aux_logits=True):
  """Creates the Inception V4 model.

  Args:
    inputs: a 4-D tensor of size [batch_size, height, width, 3].
    num_classes: number of predicted classes. If 0 or None, the logits layer
      is omitted and the input features to the logits layer (before dropout)
      are returned instead.
    is_training: whether is training or not.
    dropout_keep_prob: float, the fraction to keep before final layer.
    reuse: whether or not the network and its variables should be reused. To be
      able to reuse 'scope' must be given.
    scope: Optional compat.v1.variable_scope.
    create_aux_logits: Whether to include the auxiliary logits.

  Returns:
    net: a Tensor with the logits (pre-softmax activations) if num_classes
      is a non-zero integer, or the non-dropped input to the logits layer
      if num_classes is 0 or None.
    end_points: the set of end_points from the inception model.
  """
  end_points = {}
  with tf.compat.v1.variable_scope(
      scope, 'InceptionV4', [inputs], reuse=reuse) as scope:
    with slim.arg_scope([slim.batch_norm, slim.dropout],
                        is_training=is_training):
      net, end_points = inception_v4_base(inputs, scope=scope)

      with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                          stride=1, padding='SAME'):
        # Auxiliary Head logits
        if create_aux_logits and num_classes:
          with tf.compat.v1.variable_scope('AuxLogits'):
            # 17 x 17 x 1024
            aux_logits = end_points['Mixed_6h']
            aux_logits = slim.avg_pool2d(aux_logits, [5, 5], stride=3,
                                         padding='VALID',
                                         scope='AvgPool_1a_5x5')
            aux_logits = slim.conv2d(aux_logits, 128, [1, 1],
                                     scope='Conv2d_1b_1x1')
            aux_logits = slim.conv2d(aux_logits, 768,
                                     aux_logits.get_shape()[1:3],
                                     padding='VALID', scope='Conv2d_2a')
            aux_logits = slim.flatten(aux_logits)
            aux_logits = slim.fully_connected(aux_logits, num_classes,
                                              activation_fn=None,
                                              scope='Aux_logits')
            end_points['AuxLogits'] = aux_logits

        # Final pooling and prediction
        # TODO(sguada,arnoegw): Consider adding a parameter global_pool which
        # can be set to False to disable pooling here (as in resnet_*()).
        with tf.compat.v1.variable_scope('Logits'):
          # 8 x 8 x 1536
          kernel_size = net.get_shape()[1:3]
          if kernel_size.is_fully_defined():
            net = slim.avg_pool2d(net, kernel_size, padding='VALID',
                                  scope='AvgPool_1a')
          else:
            net = tf.reduce_mean(
                input_tensor=net,
                axis=[1, 2],
                keepdims=True,
                name='global_pool')
          end_points['global_pool'] = net
          if not num_classes:
            return net, end_points
          # 1 x 1 x 1536
          net = slim.dropout(net, dropout_keep_prob, scope='Dropout_1b')
          net = slim.flatten(net, scope='PreLogitsFlatten')
          end_points['PreLogitsFlatten'] = net
          # 1536
          logits = slim.fully_connected(net, num_classes, activation_fn=None,
                                        scope='Logits')
          end_points['Logits'] = logits
          end_points['Predictions'] = tf.nn.softmax(logits, name='Predictions')
    return logits, end_points
inception_v4.default_image_size = 299


inception_v4_arg_scope = inception_arg_scope

In [47]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# InceptionV4 Keras implementasyonu
from keras.models import Model
from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D, MaxPool2D, Input, GlobalAveragePooling2D, AveragePooling2D, Dense, Dropout, Activation, BatchNormalization

def conv2d_with_Batch(prev_layer, nbr_kernels, filter_size, strides=(1, 1), padding='valid'):
    x = Conv2D(filters=nbr_kernels, kernel_size=filter_size, strides=strides, padding=padding)(prev_layer)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    return x

def stemBlock(prev_layer):
    x = conv2d_with_Batch(prev_layer, nbr_kernels=32, filter_size=(3, 3), strides=(2, 2))
    x = conv2d_with_Batch(x, nbr_kernels=32, filter_size=(3, 3))
    x = conv2d_with_Batch(x, nbr_kernels=64, filter_size=(3, 3))

    x_1 = conv2d_with_Batch(x, nbr_kernels=96, filter_size=(3, 3), strides=(2, 2))
    x_2 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = concatenate([x_1, x_2], axis=3)

    x_1 = conv2d_with_Batch(x, nbr_kernels=64, filter_size=(1, 1))
    x_1 = conv2d_with_Batch(x_1, nbr_kernels=64, filter_size=(1, 7), padding='same')
    x_1 = conv2d_with_Batch(x_1, nbr_kernels=64, filter_size=(7, 1), padding='same')
    x_1 = conv2d_with_Batch(x_1, nbr_kernels=96, filter_size=(3, 3))

    x_2 = conv2d_with_Batch(x, nbr_kernels=96, filter_size=(1, 1))
    x_2 = conv2d_with_Batch(x_2, nbr_kernels=96, filter_size=(3, 3))

    x = concatenate([x_1, x_2], axis=3)

    x_1 = conv2d_with_Batch(x, nbr_kernels=192, filter_size=(3, 3), strides=2)
    x_2 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = concatenate([x_1, x_2], axis=3)

    return x

def reduction_A_Block(prev_layer):
    x_1 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=192, filter_size=(1, 1))
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=224, filter_size=(3, 3), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=256, filter_size=(3, 3), strides=(2, 2))

    x_2 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=384, filter_size=(3, 3), strides=(2, 2))

    x_3 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(prev_layer)

    x = concatenate([x_1, x_2, x_3], axis=3)

    return x

def reduction_B_Block(prev_layer):
    x_1 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(prev_layer)

    x_2 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=192, filter_size=(1, 1))
    x_2 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=192, filter_size=(3, 3), strides=(2, 2))

    x_3 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=256, filter_size=(1, 1))
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=256, filter_size=(1, 7), padding='same')
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=320, filter_size=(7, 1), padding='same')
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=320, filter_size=(3, 3), strides=(2, 2))

    x = concatenate([x_1, x_2, x_3], axis=3)
    return x

def InceptionBlock_A(prev_layer):
    x_1 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=64, filter_size=(1, 1))
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=96, filter_size=(3, 3), strides=(1, 1), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=96, filter_size=(3, 3), strides=(1, 1), padding='same')

    x_2 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=64, filter_size=(1, 1))
    x_2 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=96, filter_size=(3, 3), padding='same')

    x_3 = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same')(prev_layer)
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=96, filter_size=(1, 1), padding='same')

    x_4 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=96, filter_size=(1, 1))

    output = concatenate([x_1, x_2, x_3, x_4], axis=3)
    return output

def InceptionBlock_B(prev_layer):
    x_1 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=192, filter_size=(1, 1))
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=192, filter_size=(7, 1), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=224, filter_size=(1, 7), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=224, filter_size=(7, 1), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=256, filter_size=(1, 7), padding='same')

    x_2 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=192, filter_size=(1, 1))
    x_2 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=224, filter_size=(1, 7), padding='same')
    x_2 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=256, filter_size=(7, 1), padding='same')

    x_3 = AveragePooling2D(pool_size=(3, 3), strides=1, padding='same')(prev_layer)
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=128, filter_size=(1, 1))

    x_4 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=384, filter_size=(1, 1))

    output = concatenate([x_1, x_2, x_3, x_4], axis=3)
    return output

def InceptionBlock_C(prev_layer):
    x_1 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=384, filter_size=(1, 1))
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=448, filter_size=(3, 1), padding='same')
    x_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=512, filter_size=(1, 3), padding='same')
    x_1_1 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=256, filter_size=(1, 3), padding='same')
    x_1_2 = conv2d_with_Batch(prev_layer=x_1, nbr_kernels=256, filter_size=(3, 1), padding='same')
    x_1 = concatenate([x_1_1, x_1_2], axis=3)

    x_2 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=384, filter_size=(1, 1))
    x_2_1 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=256, filter_size=(1, 3), padding='same')
    x_2_2 = conv2d_with_Batch(prev_layer=x_2, nbr_kernels=256, filter_size=(3, 1), padding='same')
    x_2 = concatenate([x_2_1, x_2_2], axis=3)

    x_3 = MaxPool2D(pool_size=(3, 3), strides=1, padding='same')(prev_layer)
    x_3 = conv2d_with_Batch(prev_layer=x_3, nbr_kernels=256, filter_size=(1, 1), padding='same')

    x_4 = conv2d_with_Batch(prev_layer=prev_layer, nbr_kernels=256, filter_size=(1, 1))

    output = concatenate([x_1, x_2, x_3, x_4], axis=3)
    return output

def InceptionV4():
    input_layer = Input(shape=(299, 299, 3))

    x = stemBlock(prev_layer=input_layer)

    x = InceptionBlock_A(prev_layer=x)
    x = InceptionBlock_A(prev_layer=x)
    x = InceptionBlock_A(prev_layer=x)
    x = InceptionBlock_A(prev_layer=x)

    x = reduction_A_Block(prev_layer=x)

    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)
    x = InceptionBlock_B(prev_layer=x)

    x = reduction_B_Block(prev_layer=x)

    x = InceptionBlock_C(prev_layer=x)
    x = InceptionBlock_C(prev_layer=x)
    x = InceptionBlock_C(prev_layer=x)

    # Özel bir sınıflandırıcı ekle
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=1024, activation='relu')(x)
    predictions = Dense(1)(x)  # Tek bir çıktı nöronu (yaş tahmini)

    model = Model(inputs=input_layer, outputs=predictions, name='Inception-V4')

    return model

# --- Ana Kod ---

# GPU Ayarları
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Her bir GPU için bellek büyümesini ayarla
        for gpu in physical_devices:
            print()
            # tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU'lar kullanılıyor.")
    except RuntimeError as e:
        print(f"GPU ayarlama hatası: {e}")
else:
    print("GPU bulunamadı. CPU kullanılacak.")

# Veri seti yolu (Ana klasör yolu)
veri_seti_yolu = "/kaggle/input/miniddsm/Mini_DDSM_Upload"

# Alt klasörler
alt_klasorler = ["Benign", "Cancer", "Normal"]

# Tüm verileri bir listede birleştir
tum_veriler = []

# Her alt klasör için veriyi yükle ve birleştir
for alt_klasor in alt_klasorler:
    excel_yolu = os.path.join(veri_seti_yolu, f"List_{alt_klasor}.xlsx")
    df = pd.read_excel(excel_yolu)
    # Görüntü yollarını oluştururken alt klasörü de ekleyin
    df['imagePath'] = df['fileName'].apply(lambda x: os.path.join(veri_seti_yolu, alt_klasor, x))
    tum_veriler.append(df)

# Tüm veri çerçevelerini birleştir
df = pd.concat(tum_veriler, ignore_index=True)

# Veriyi eğitim ve test kümelerine ayır
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Görüntü boyutları
img_width, img_height = 220, 360  # InceptionV4 için giriş boyutu

# InceptionV4 modelini oluştur
model = InceptionV4()

# Modelin son katmanlarını eğitmek için diğer katmanları dondur
for layer in model.layers[:-2]:  # Son iki katman hariç
    layer.trainable = False

# Modeli derle
model.compile(optimizer='adam', loss='mean_absolute_error')

# Veri artırma
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # InceptionV4 için gerekli önişleme
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Veri yükleyicilerini oluştur
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='imagePath',
    y_col='Age',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='raw')  # Çıktı sürekli bir değer olduğu için "raw" kullanılır

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='imagePath',
    y_col='Age',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='raw')

# Modeli eğit (RAM kullanımını azaltmak için küçük bir epoch sayısı)
model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator)

# Özellik vektörlerini al
features = model.predict(test_generator)

# KNN regresörü
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(features, test_df['Age'])

# Tahminler
knn_predictions = knn.predict(features)
 
# Performans ölçümleri
mae = mean_absolute_error(test_df['Age'], knn_predictions)
mse = mean_squared_error(test_df['Age'], knn_predictions)

print(f"Ortalama Mutlak Hata (MAE): {mae}")
print(f"Karesel Ortalama Hata (MSE): {mse}")

# Yeni bir resim için yaş tahmini yapma fonksiyonu
def yas_tahmin_et(resim_yolu):
    # Resmi yükle ve önişle
    img = load_img(resim_yolu, target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)  # InceptionV4 için gerekli önişleme
       
    # Özellik vektörünü al
    features = model.predict(x)

    # Yaş tahmini yap
    yas_tahmini = knn.predict(features)

    return yas_tahmini[0]
 
# Tahmin edilecek resmin yolu
test_resim_yolu = "/kaggle/input/miniddsm/Mini_DDSM_Upload/Normal/1000_A_0358_1.RIGHT_CC.LJPEG.1_highpass.png"  # Örnek resim yolu

# Yaş tahminini yap
tahmin_edilen_yas = yas_tahmin_et(test_resim_yolu)

print(f"Tahmin Edilen Yaş: {tahmin_edilen_yas:.0f}")



GPU'lar kullanılıyor.
Found 7747 validated image filenames.
Found 1937 validated image filenames.
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1716042987.568979     640 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716042987.694101     640 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


127/243 ━━━━━━━━━━━━━━━━━━━━ 2:11 1s/step - loss: 56.6656

W0000 00:00:1716043130.800038     639 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 53.1448

...

243/243 ━━━━━━━━━━━━━━━━━━━━ 236s 945ms/step - loss: 12.0340 - val_loss: 10.6893
Epoch 3/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 238s 952ms/step - loss: 10.8976 - val_loss: 10.6801
Epoch 4/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 236s 946ms/step - loss: 10.9443 - val_loss: 10.6857
Epoch 5/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 233s 932ms/step - loss: 10.8740 - val_loss: 10.6558
Epoch 6/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 234s 941ms/step - loss: 11.1274 - val_loss: 10.6565
Epoch 7/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 238s 951ms/step - loss: 10.8004 - val_loss: 10.6272
Epoch 8/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 239s 956ms/step - loss: 10.8202 - val_loss: 10.6182
Epoch 9/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 237s 948ms/step - loss: 10.8321 - val_loss: 10.5949
Epoch 10/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 238s 952ms/step - loss: 10.8634 - val_loss: 10.5920
Epoch 11/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 239s 960ms/step - loss: 10.8455 - val_loss: 10.5694
Epoch 12/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 238s 951ms/step - loss: 10.7350 - val_loss: 10.5

W0000 00:00:1716054946.306038     637 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


61/61 ━━━━━━━━━━━━━━━━━━━━ 34s 390ms/step


W0000 00:00:1716054969.721859     637 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Ortalama Mutlak Hata (MAE): 9.19659266907589
Karesel Ortalama Hata (MSE): 125.16173464119773
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
Tahmin Edilen Yaş: 62
